In [202]:

import torch
from torch.nn.functional import normalize
import pickle 
import torch.nn.functional as F 
import numpy as np 
from sklearn.metrics import f1_score
from torch import nn, optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pickle
import random
from scipy import sparse
import itertools
from scipy.io import savemat, loadmat
import re
import string
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import pandas as pd
import sys
sys.path.insert(1, '/home/ec2-user/SageMaker/github/aspect_topic_modeling')
import swifter
from src.models.utils import remove_stopWords
from src.models.utils import get_wordnet_pos, remove_stopWords, get_emb, generate_emb, train, kld_normal, get_common_words, generate_bow
from collections import Counter
from models.NVDM import topic_covariance_penalty, sinkhorn_torch, NTM, negative_sampling_prior, optimal_transport_prior,  NormalParameter, get_mlp, EmbTopic, NSSTM, OTETM
from src.models.utils import get_wordnet_pos, remove_stopWords, get_emb, generate_emb, train, kld_normal, get_common_words
from hyperspherical_vae.distributions import VonMisesFisher
from hyperspherical_vae.distributions import HypersphericalUniform
from src.features.metric import diversity, get_topic_coherence

In [75]:
!pwd


/home/ec2-user/SageMaker/github/aspect_topic_modeling/notebooks


# 20News

In [915]:
# 20News Data
print('reading data...')
data = fetch_20newsgroups(subset = 'all')

reading data...


In [11]:
data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [916]:
df = pd.DataFrame()
df['class'] = data['target']
df['text'] = data['data']

In [18]:
df.shape

(18846, 2)

In [917]:
df['text'] = df.swifter.apply(lambda x: ' '.join(remove_stopWords(x['text'])), axis=1)
df['clean_text']  = df.swifter.apply(lambda x: x['text'].split(), axis = 1)

common_words_ct = Counter([j for i in df['clean_text'].values for j in i])
common_words = get_common_words(common_words_ct, ct = 200)
word_track = {i: ind for ind, i in enumerate(common_words)}
index_track = {ind: i for ind, i in enumerate(common_words)}
df['index_num'] = df.apply(
            lambda x: [word_track[i] for i in x['clean_text'] if i in word_track], axis=1)


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/18846 [00:00<?, ?it/s]

In [26]:
np.savetxt('/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/20news/text.txt',
           df['text'].values,
           fmt='%s')

In [29]:
#change it to any location you save your embeddings
vec = '/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/20news/jose.txt'
embed = generate_emb(vec, common_words).cpu()
X, indices = generate_bow(df = df, common_words = common_words)


In [918]:
len(common_words)

2394

In [921]:
[i for i in common_words if i not in ENGLISH_STOP_WORDS][:100]

['andrew',
 'cmu',
 'edu',
 'subject',
 'pen',
 'fan',
 'reaction',
 'organization',
 'post',
 'office',
 'carnegie',
 'mellon',
 'pittsburgh',
 'line',
 'nntp',
 'host',
 'sure',
 'pretty',
 'confuse',
 'lack',
 'kind',
 'recent',
 'massacre',
 'devil',
 'actually',
 'bit',
 'end',
 'non',
 'man',
 'kill',
 'bad',
 'think',
 'good',
 'regular',
 'season',
 'stats',
 'lot',
 'fun',
 'watch',
 'playoff',
 'let',
 'couple',
 'game',
 'beat',
 'lose',
 'final',
 'rule',
 'midway',
 'ecn',
 'uoknor',
 'matthew',
 'high',
 'performance',
 'vlb',
 'video',
 'card',
 'summary',
 'seek',
 'engineering',
 'computer',
 'network',
 'university',
 'usa',
 'keywords',
 'brother',
 'market',
 'support',
 'vesa',
 'local',
 'bus',
 'ram',
 'suggestion',
 'idea',
 'diamond',
 'pro',
 'ati',
 'graphic',
 'email',
 'thank',
 'matt',
 'king',
 'heaven',
 'right',
 'way',
 'armenia',
 'say',
 'shoot',
 'turkish',
 'plane',
 'reply',
 'dept',
 'science',
 'student',
 'write',
 'great',
 'black',
 'sea',
 '

In [919]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
len([i for i in common_words if i not in ENGLISH_STOP_WORDS])



2272

# TWN

In [908]:
df1.head()

,text,edge,intent
0,asian export fear damag japan rift mount trade...,asian export fear damag japan rift mount trade...,trade
1,china daili vermin eat pct grain stock survei ...,china daili vermin eat pct grain stock survei ...,grain
2,australian foreign ship ban end nsw port hit t...,australian foreign ship ban end nsw port hit t...,ship
3,sumitomo bank aim quick recoveri merger sumito...,sumitomo bank aim quick recoveri merger sumito...,acq
4,amatil propos two for bonu share issu amatil a...,amatil propos two for bonu share issu amatil a...,earn


In [841]:
df1 = pd.read_csv('/home/ec2-user/SageMaker/github/aspect_topic_modeling/data/external/r8-test-stemmed.csv')
df2 = pd.read_csv('/home/ec2-user/SageMaker/github/aspect_topic_modeling/data/external/r8-train-stemmed.csv')
df = df1.append(df2)
df['clean_text']  = df.swifter.apply(lambda x: x['text'].split(), axis = 1)
common_words_ct = Counter([j for i in df['clean_text'].values for j in i])
common_words = get_common_words(common_words_ct, ct = 200)
word_track = {i: ind for ind, i in enumerate(common_words)}
index_track = {ind: i for ind, i in enumerate(common_words)}
df['index_num'] = df.apply(
            lambda x: [word_track[i] for i in x['clean_text'] if i in word_track], axis=1)
df['class'] = df['intent']

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/swifter/swifter.py:37: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  "This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`."


Pandas Apply:   0%|          | 0/7126 [00:00<?, ?it/s]

In [837]:
np.savetxt('/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/R8/text.txt',
           df['text'].values,
           fmt='%s')

In [842]:
#change it to any location you save your embeddings
vec = '/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/R8/jose.txt'
embed = generate_emb(vec, common_words).cpu()
X, indices = generate_bow(df = df, common_words = common_words)


# AgNews

In [911]:
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [940]:
common_words_ct.most_common(100)

[('the', 240794),
 ('be', 180408),
 ('a', 127679),
 ('to', 124248),
 ('of', 114080),
 ('and', 95818),
 ('i', 87457),
 ('in', 81670),
 ('that', 65129),
 ('ax', 62473),
 ('it', 59120),
 ('have', 51881),
 ('for', 46520),
 ('you', 44592),
 ('from', 37389),
 ('s', 36439),
 ('edu', 35703),
 ('on', 33701),
 ('this', 32813),
 ('not', 31746),
 ('do', 30519),
 ('t', 30325),
 ('with', 28671),
 ('or', 24298),
 ('1', 23600),
 ('if', 22625),
 ('but', 21995),
 ('line', 21842),
 ('they', 21778),
 ('subject', 20535),
 ('com', 20223),
 ('can', 20097),
 ('m', 19432),
 ('organization', 18952),
 ('at', 18496),
 ('by', 18343),
 ('re', 17622),
 ('2', 16450),
 ('what', 16449),
 ('an', 16420),
 ('my', 15915),
 ('there', 15886),
 ('one', 15389),
 ('all', 15318),
 ('would', 14825),
 ('we', 14713),
 ('0', 14691),
 ('write', 14659),
 ('will', 14640),
 ('use', 14286),
 ('3', 14169),
 ('he', 13783),
 ('about', 13765),
 ('get', 13476),
 ('x', 13347),
 ('so', 12698),
 ('article', 12694),
 ('no', 12623),
 ('your', 1231

In [942]:
X.shape[0] * 0.15

8188.95

In [945]:
'good' in common_words

True

In [926]:
common_words = [i for i in common_words if i not in ENGLISH_STOP_WORDS]

In [924]:
len([ i for i in common_words if i not in stopwords.words('english')])

2381

In [ ]:
#data import
df = pd.read_csv('https://raw.githubusercontent.com/yumeng5/WeSTClass/master/agnews/dataset.csv', 
                   error_bad_lines=False,
                   names = ['class', 'title', 'description'])
df['text'] = df.apply(lambda x: ' '.join(remove_stopWords(x['title'] + x['description'])), axis=1)
df['clean_text']  = df.apply(lambda x: x['text'].split(), axis = 1)
#get clean data
common_words_ct = Counter([j for i in df['clean_text'].values for j in i])
common_words = get_common_words(common_words_ct, ct = 100)
word_track = {i: ind for ind, i in enumerate(common_words)}
index_track = {ind: i for ind, i in enumerate(common_words)}
df['index_num'] = df.apply(
            lambda x: [word_track[i] for i in x['clean_text'] if i in word_track], axis=1)


In [ ]:
#change it to any location you save your embeddings
vec = '/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/agnews/jose.txt'
embed = generate_emb(vec, common_words).cpu()
X, indices = generate_bow(df = df, common_words = common_words)


In [909]:
news.head()

,text,train,class,clean_text,index_num
0,fast cut protocol agent coordination,train,1,"[fast, cut, protocol, agent, coordination]","[0, 1, 2, 3, 4]"
1,retrieval base class svm,train,1,"[retrieval, base, class, svm]","[5, 6, 7, 8]"
2,semantic annotation personal video content image,train,2,"[semantic, annotation, personal, video, conten...","[9, 10, 11, 12, 13, 14]"
3,semantic repository modeling image database,train,2,"[semantic, repository, modeling, image, database]","[9, 15, 14, 16]"
4,global local scheme imbalanced point matching,train,2,"[global, local, scheme, imbalanced, point, mat...","[17, 18, 19, 20, 21]"


In [891]:
news = pd.read_csv('/home/ec2-user/SageMaker/github/aspect_topic_modeling/data/external/corpusbibitex.tsv',  sep = '\t',
                   error_bad_lines=False,
                   names = ['text', 'train', 'class'])
#change it to any location you save your embeddings
#news = news.groupby('class').head(4500)
news['clean_text']  = news.apply(lambda x: x['text'].split(), axis = 1)
#get clean data
common_words_ct = Counter([j for i in news['clean_text'].values for j in i])
common_words = get_common_words(common_words_ct, ct = 50) #this vocab
word_track = {i: ind for ind, i in enumerate(common_words)} #word dict
index_track = {ind: i for ind, i in enumerate(common_words)} 
news['index_num'] = news.swifter.apply(
            lambda x: [word_track[i] for i in x['clean_text'] if i in word_track], axis=1)

vec = '/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/bibi/jose.txt'
embed = generate_emb(vec, common_words).cpu()
X, indices = generate_bow(df = news, common_words = common_words)

Pandas Apply:   0%|          | 0/54595 [00:00<?, ?it/s]

# Metrics

In [56]:
>>> from sklearn import metrics
>>> labels_true = [1, 1, 1, 0, 0, 0]
>>> labels_pred = [0, 2, 2, 2, 2, 2]

>>> metrics.mutual_info_score(labels_true, labels_pred)  



0.13230412471889824

In [47]:
labels_pred = [3, 3, 1, 1, 2, 2]
metrics.mutual_info_score(labels_true, labels_pred)  


0.4620981203732969

In [203]:
from collections import Counter
 
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][1]
   
List = [2, 1, 2, 2, 1, 3]
print(most_frequent(List))

3


In [271]:
from collections import defaultdict
def top_purity(labels_true, labels_pred):
    pred = np.unique(labels_pred)
    d = defaultdict(list)
    for i, j in zip(labels_pred, labels_true):
        d[i].append(j)
    ct = []
    for i in d:
        ct += [Counter(d[i]).most_common(1)[0][1]]
        #print(Counter(d[i]).most_common(1)[0], i, len(d[i]))
    print(ct)
    return np.sum(ct)/len(labels_pred)
        

In [270]:
top_purity(labels_true, labels_pred)

[952, 629, 18, 23, 9, 10, 1, 1, 1, 1, 1]


1646

# VNTM

In [611]:
def topic_embedding_weighted_penalty(embedding_weight, topic_word_logit, EPS=1e-12):
    """embedding_weight: V x dim, topic_word_logit: T x V."""
    #add importance to each word
    w = topic_word_logit.transpose(0, 1)  # V x T
    #normalized embeddings
    nv = embedding_weight / (torch.norm(embedding_weight, dim=1, keepdim=True) + EPS)  # V x dim
    #normalized importance
    nw = w / (torch.norm(w, dim=0, keepdim=True) + EPS)  # V x T
    #get topic representation
    t = nv.transpose(0, 1) @ w  # dim x T
    nt = t / (torch.norm(t, dim=0, keepdim=True) + EPS)  # dim x T
    #word - topics similarity matrix
    s = nv @ nt  # V x T
    #we want normalized importance is closed their similarity
    return -(s * nw).sum()  # minus for minimization


In [708]:
class VNTM(nn.Module):
    """NTM that keeps track of output
    """
    def __init__(self, hidden, normal, h_to_z, topics, layer, top_number, penalty):
        super(VNTM, self).__init__()
        self.hidden = hidden
        #self.normal = normal
        self.h_to_z = h_to_z
        self.topics = topics
        self.output = None
        self.drop = nn.Dropout(p=0.5)
        self.fc_mean = nn.Linear(layer, top_number)
        self.fc_var = nn.Linear(layer, 1)
        self.num = top_number
        self.penalty = penalty
        #self.dirichlet = torch.distributions.dirichlet.Dirichlet((torch.ones(self.topics.k)/self.topics.k).cuda())
    def forward(self, x, n_sample=1, epoch = 0):
        h = self.hidden(x)
        h = self.drop(h)
        z_mean = self.fc_mean(h)
        z_mean = z_mean / z_mean.norm(dim=-1, keepdim=True)
        # the `+ 1` prevent collapsing behaviors
        z_var = F.softplus(self.fc_var(h)) + 1
        
        q_z = VonMisesFisher(z_mean, z_var)
        p_z = HypersphericalUniform(self.num - 1, device=device)
        kld = torch.distributions.kl.kl_divergence(q_z, p_z).mean()
        #print(q_z)
        #mu, log_sigma = self.normal(h)
        #identify how far it is away from normal distribution
        
        #print(kld.shape)
        rec_loss = 0
        for i in range(n_sample):
            #reparametrician trick
            z = q_z.rsample()
            #z = nn.Softmax()(z)
            #decode
            #print(z)
            
            z = self.h_to_z(10 * z)
            self.output = z
            #print(z)
            
            #get log probability for reconstruction loss
            log_prob = self.topics(z)
            rec_loss = rec_loss - (log_prob * x).sum(dim=-1)
        #average reconstruction loss
        rec_loss = rec_loss / n_sample
        #print(rec_loss.shape)
        minus_elbo = rec_loss + kld
        penalty, var, mean = topic_covariance_penalty(self.topics.topic_emb) 
        return {
            'loss': minus_elbo + penalty * self.penalty,
            'minus_elbo': minus_elbo,
            'rec_loss': rec_loss,
            'kld': kld
        }

    def get_topics(self):
        return self.topics.get_topics()


In [608]:
model.topics.get_topic_word_logit()

ModuleAttributeError: 'EmbTopic' object has no attribute 'get_topic_word_logit'

In [709]:
def train(model, X, batch_size, epoch, optimizer, scheduler):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    total_nll = 0.0
    total_kld = 0.0
    total_words = 0
    total_penalty = 0.0
    #size = epoch_size * batch_size
    indices = torch.randperm(X.shape[0])
    indices = torch.split(indices, batch_size)
    #print(indices)
    length = len(indices)
    for idx, ind in enumerate(indices):
        data_batch = torch.from_numpy(X[ind].toarray()).float().to(device)
        
        d = model(x = data_batch)
            
        
        
        total_nll += d['rec_loss'].sum().item() / batch_size
        total_kld += d['kld'].sum().item() / batch_size  
        #total_penalty += d['prior']  
#         if i < 3:
#             loss = d['minus_elbo']
#         else:
        loss = d['loss']

        optimizer.zero_grad()
        loss.sum().backward()
        optimizer.step()
        scheduler.step()
    print(total_nll/length, total_kld/length)
     


In [853]:
X.shape

(7126, 363)

In [905]:
            gc.collect()
            numb_embeddings = 20
            hidden = get_mlp([X.shape[1], 128], nn.ReLU)
            normal = NormalParameter(128, 20)
            h_to_z = nn.Softmax()
            embedding = nn.Embedding(X.shape[1], 20)
            # p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
            # out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

            embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
            embedding.weight.requires_grad=False
            topics = EmbTopic(embedding = embedding,
                              k = numb_embeddings, normalize = False)



            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


            model1 = VNTM(hidden = hidden,
                        normal = normal,
                        h_to_z = h_to_z,
                        topics = topics,
                        layer = 128, 
                        top_number = numb_embeddings,
                        penalty = 1
                        ).to(device).float()
            # larger hidden size make topics more diverse
            #num_docs_train = 996318
            batch_size = 256
            optimizer = optim.Adam(model1.parameters(), 
                                   lr=0.002, 
                                   weight_decay=1.2e-6)

            epochs = 20
            scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.05, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

            
            for epoch in range(epochs):
                train(model1, X,  batch_size, epoch, optimizer, scheduler)


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


35.631355989759214 0.00018478308840509
34.5023542163528 0.00018801333730339691
33.57135835540629 0.0009436759800991809
32.72929679567569 0.0021514606588111025
32.329653472543875 0.002785144748044348
32.10514688046179 0.0032073157278072334
31.947803755786932 0.003495215552545666
31.87388635795807 0.0036559811353718288
31.81700792045237 0.0038143141558567915


KeyboardInterrupt: 

In [ ]:
from scipy.optimize import linear_sum_assignment


In [898]:
df = news.iloc[indices]
df.iloc[indices].head()

IndexError: positional indexers are out-of-bounds

In [906]:
#prior(softmax_top, indexes)
emb = model1.topics.get_topics().cpu().detach().numpy()
topics =  [[index_track[ind] for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(numb_embeddings)]
data_batch = torch.from_numpy(X.toarray()).float()
model1.cpu()
z = model1.hidden(data_batch)
z_mean = model1.fc_mean(z)
z_mean = z_mean / z_mean.norm(dim=-1, keepdim=True)
z = model1.h_to_z(z_mean)


labels_pred = torch.argmax(z, 1).numpy()
labels_true = df['class'].values
 

diversity_score = np.mean(diversity(topics))
#coherence_score = get_topic_coherence(X.toarray(), topics, word_track)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=numb_embeddings, random_state=0).fit(z.detach().numpy())
top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)  
            

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[420, 2390, 288, 2442, 880, 4503, 3407, 2868, 3173, 476, 4223, 3938, 667, 372, 3554, 530, 136, 166, 78, 152]
[2036, 3403, 1775, 3294, 2615, 2564, 1066, 1096, 2676, 829, 852, 1047, 2566, 601, 2996, 666, 1068, 2476, 790, 768]


(0.634934881761398,
 0.17736129097988454,
 0.6444782298096826,
 0.17401671830334664)

In [907]:
topics =  [[index_track[ind] for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(numb_embeddings)]
sorted(diversity(topics)), np.mean(diversity(topics))

([0.0,
  0.0,
  0.04,
  0.04,
  0.04,
  0.04,
  0.04,
  0.08,
  0.08,
  0.36,
  0.48,
  0.48,
  0.52,
  0.56,
  0.6,
  0.64,
  0.68,
  0.72,
  0.8,
  0.84],
 0.352)

In [473]:
from sklearn.decomposition import LatentDirichletAllocation, NMF

lda = LatentDirichletAllocation(n_components=20, random_state=0)
lda.fit(X)
topic_index = np.argsort(lda.components_, axis=1)[:, -25:]
topics = [[index_track[j] for j in i] for i in topic_index]
np.mean(diversity(topics)), np.mean(get_topic_coherence(X.toarray(), topics, word_track))

(0.5040000000000001, -0.40466082922081026)

In [860]:
topics


[['stg',
  'rise',
  'final',
  'turnov',
  'plc',
  'improv',
  'strong',
  'payabl',
  'div',
  'pretax',
  'intern',
  'extraordinari',
  'dilut',
  'show',
  'adjust',
  'subject',
  'nil',
  'incom',
  'result',
  'posit',
  'properti',
  'make',
  'cent',
  'european',
  'long'],
 ['year',
  'dlr',
  'profit',
  'mln',
  'billion',
  'earn',
  'net',
  'compar',
  'quarter',
  'sale',
  'revenu',
  'loss',
  'oper',
  'estim',
  'last',
  'expect',
  'earlier',
  'result',
  'compani',
  'ago',
  'fiscal',
  'asset',
  'provis',
  'total',
  'incom'],
 ['york',
  'system',
  'cost',
  'energi',
  'area',
  'revenu',
  'estim',
  'servic',
  'monei',
  'incom',
  'file',
  'fiscal',
  'fourth',
  'offic',
  'manag',
  'provid',
  'outstand',
  'line',
  'depart',
  'feder',
  'texa',
  'account',
  'western',
  'recent',
  'problem'],
 ['share',
  'inc',
  'offer',
  'common',
  'corp',
  'sharehold',
  'acquir',
  'compani',
  'outstand',
  'bui',
  'group',
  'stock',
  'dlr',
 

# NTM

In [232]:
def train(model, X, batch_size, epoch, optimizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    total_nll = 0.0
    total_kld = 0.0
    total_words = 0
    total_penalty = 0.0
    #size = epoch_size * batch_size
    indices = torch.randperm(X.shape[0])
    indices = torch.split(indices, batch_size)
    #print(indices)
    length = len(indices)
    for idx, ind in enumerate(indices):
        data_batch = torch.from_numpy(X[ind].toarray()).float().to(device)
        
        d = model(x = data_batch)
            
        
        
        total_nll += d['rec_loss'].sum().item() / batch_size
        total_kld += d['kld'].sum().item() / batch_size  
        #total_penalty += d['penalty'].sum().item() / batch_size  
#         if i < 3:
#             loss = d['minus_elbo']
#         else:
        loss = d['loss']

        optimizer.zero_grad()
        loss.sum().backward()
        optimizer.step()

    print(total_nll/length, total_kld/length)

In [249]:
topics = Topics(numb_embeddings, X.shape[1])
topics.get_topics()

tensor([[0.0005, 0.0005, 0.0004,  ..., 0.0003, 0.0004, 0.0005],
        [0.0005, 0.0004, 0.0004,  ..., 0.0004, 0.0005, 0.0004],
        [0.0003, 0.0003, 0.0005,  ..., 0.0005, 0.0005, 0.0005],
        ...,
        [0.0004, 0.0005, 0.0003,  ..., 0.0004, 0.0005, 0.0004],
        [0.0004, 0.0005, 0.0005,  ..., 0.0003, 0.0004, 0.0004],
        [0.0005, 0.0004, 0.0004,  ..., 0.0005, 0.0004, 0.0004]])

In [251]:
from models.NVDM import NTM
import gc
#labels = [[120], [1527], [1646], [2047], [727], [1624], [36], [32], [26], [92], [907], [652]]
numb_embeddings = 20
torch.cuda.empty_cache()
gc.collect()
hidden = get_mlp([X.shape[1], 64], nn.ReLU)
normal = NormalParameter(64, numb_embeddings)
h_to_z = nn.Softmax()
embedding = nn.Embedding(X.shape[1], 50)
# p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
# out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

# embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
# embedding.weight.requires_grad=False
topics = Topics(numb_embeddings, X.shape[1])



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = NTM(hidden = hidden,
            normal = normal,
            h_to_z = h_to_z,
            topics = topics
            ).to(device).float()
# larger hidden size make topics more diverse
#num_docs_train = 996318
batch_size = 256
optimizer = optim.Adam(model.parameters(), 
                       lr=0.002, 
                       weight_decay=1.2e-6)



epochs = 20
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.05, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

for epoch in range(epochs):
    train(model, X,  batch_size, epoch, optimizer)


836.3474549989443 0.1607032038548307
819.3788678968275 0.03806465583526202
806.9361918681377 0.03929201230985691
797.9546310837204 0.048760768302993196
791.4055390229096 0.04600683428548478
786.7550077696104 0.038150564075221084
783.4681206780512 0.02321837155299412
781.0415558686127 0.025440513194110746
779.4308673755543 0.04452345746481237
778.4024130331503 0.0753821546048228
777.6905938225824 0.06864226998987834
777.3639212943413 0.023134106206959364
776.9615222827808 0.1023010356647491
776.8203562143686 0.06381778039881406
776.6001384838207 0.0499157341703697
776.7200680294552 0.10331941790236915
776.361110377956 0.13221442471793224
776.5740822456978 0.11337339315198462
776.1798528722815 0.162217427351877
776.2847937506598 0.10506580079704322


In [272]:
h = model.hidden(data_batch)
h = model.drop(h)
mu, log_sigma = model.normal(h)
z = model.h_to_z(mu)
labels_pred = torch.argmax(z, 1).numpy()
labels_true = df['class'].values
diversity_score = np.mean(diversity(topics))
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(z.detach().numpy())
top_purity(labels_true, labels_pred), metrics.mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.mutual_info_score(labels_true, kmeans.labels_)  
            

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[964, 624, 19, 21, 9, 6, 1, 1, 1, 1]
[981, 67, 121, 14, 51, 58, 81, 120, 14, 8, 18, 4, 21, 12, 29, 7, 23, 4, 13, 5]


(0.08739255014326648,
 0.09698613336289995,
 0.08760479677385122,
 0.12068967614918263)

[0.64, 0.64, 0.72, 1.0, 0.84, 1.0, 0.88, 0.68, 0.76, 0.92]

array([list(['from', 'mamatha', 'devineni', 'ratnam', 'mr47', 'andrew', 'cmu', 'edu', 'subject', 'pen', 'fan', 'reaction', 'organization', 'post', 'office', 'carnegie', 'mellon', 'pittsburgh', 'pa', 'line', '12', 'nntp', 'post', 'host', 'po4', 'andrew', 'cmu', 'edu', 'i', 'be', 'sure', 'some', 'bashers', 'of', 'pen', 'fan', 'be', 'pretty', 'confuse', 'about', 'the', 'lack', 'of', 'any', 'kind', 'of', 'post', 'about', 'the', 'recent', 'pen', 'massacre', 'of', 'the', 'devil', 'actually', 'i', 'be', 'bit', 'puzzled', 'too', 'and', 'a', 'bit', 'relieved', 'however', 'i', 'be', 'go', 'to', 'put', 'an', 'end', 'to', 'non', 'pittsburghers', 'relief', 'with', 'a', 'bit', 'of', 'praise', 'for', 'the', 'pen', 'man', 'they', 'be', 'kill', 'those', 'devil', 'bad', 'than', 'i', 'think', 'jagr', 'just', 'show', 'you', 'why', 'he', 'be', 'much', 'good', 'than', 'his', 'regular', 'season', 'stats', 'he', 'be', 'also', 'a', 'lot', 'fo', 'fun', 'to', 'watch', 'in', 'the', 'playoff', 'bowman', 'should', 

# GSM

In [427]:
class GSM(NTM):
    def __init__(self, hidden, normal, h_to_z, topics, penalty):
        # h_to_z will output probabilities over topics
        super(GSM, self).__init__(hidden, normal, h_to_z, topics)
        self.penalty = penalty

    def forward(self, x, n_sample=1):
        stat = super(GSM, self).forward(x, n_sample)
        loss = stat['loss']
        penalty, var, mean = topic_covariance_penalty(self.topics.topic_emb)

        stat.update({
            'loss': loss + penalty * self.penalty,
            'penalty_mean': mean,
            'penalty_var': var,
            'penalty': penalty * self.penalty,
        })

        return stat

In [435]:
embed.shape

torch.Size([2394, 50])

In [880]:
from models.NVDM import NTM
import gc
#labels = [[120], [1527], [1646], [2047], [727], [1624], [36], [32], [26], [92], [907], [652]]
numb_embeddings = 10
torch.cuda.empty_cache()
gc.collect()
hidden = get_mlp([X.shape[1], 64], nn.ReLU)
normal = NormalParameter(64, numb_embeddings)
h_to_z = nn.Softmax()
embedding = nn.Embedding(X.shape[1], 50)
# p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
# out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

embedding.weight = torch.nn.Parameter(torch.ones(embed.shape))
# embedding.weight.requires_grad=False
#embedding.weight = torch.nn.Parameter()
embedding.weight.requires_grad=True
topics = EmbTopic(embedding = embedding,
                  
                  k = numb_embeddings, normalize = False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = GSM(hidden = hidden,
            normal = normal,
            h_to_z = h_to_z,
            topics = topics,
            penalty = 10
        
            ).to(device).float()
# larger hidden size make topics more diverse
#num_docs_train = 996318
batch_size = 256
optimizer = optim.Adam(model.parameters(), 
                       lr=0.002, 
                       weight_decay=1.2e-6)



epochs = 20
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

for epoch in range(epochs):
    train(model, X,  batch_size, epoch, optimizer, scheduler)


/home/ec2-user/SageMaker/github/aspect_topic_modeling/models/NVDM.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = self.h_to_z(z)


215.70148304530554 0.14776690251060895
214.76792471749442 0.1005628505455596
211.87730516706193 0.23094200475939683
205.10287530081612 0.9738367817231587
195.9407343183245 1.8370886487620217
190.70264598301478 2.188159533909389
188.08613804408483 2.5685715675354004
186.05395344325476 3.0157410332134793
184.89676230294364 3.212756190981184
184.2214072091239 3.352269853864397
183.68893814086914 3.470014682837895
183.20615713936942 3.5006412523133412
182.8960827418736 3.618808550494058
182.43153599330358 3.6915909137044634
182.3068389892578 3.723042828696115
182.09178924560547 3.8733092716761996
181.99352645874023 3.77488249540329
182.0485714503697 3.7913839391299655
181.976445879255 3.786278716155461
181.8711782182966 3.839538880756923


In [881]:
#prior(softmax_top, indexes)
emb = model.topics.get_topics().cpu().detach().numpy()
topics =  [[index_track[ind] for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(numb_embeddings)]
data_batch = torch.from_numpy(X.toarray()).float()
model.cpu()
h = model.hidden(data_batch)
h = model.drop(h)
mu, log_sigma = model.normal(h)
z = model.h_to_z(mu)
labels_pred = torch.argmax(z, 1).numpy()
labels_true = df['class'].values
diversity_score = np.mean(diversity(topics))
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(z.detach().numpy())
top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)  
            

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[333, 295, 1755, 2474, 53, 672, 4, 4]
[90, 454, 95, 210, 243, 332, 266, 80, 518, 70, 345, 214, 70, 239, 620, 679, 509, 239, 154, 194]


(0.7844513050799887,
 0.49100081796451894,
 0.7888015717092338,
 0.3558329818145496)

In [882]:
np.mean(diversity(topics))


0.22000000000000003

In [883]:
from models.NVDM import NTM
import gc
#labels = [[120], [1527], [1646], [2047], [727], [1624], [36], [32], [26], [92], [907], [652]]
numb_embeddings = 20
torch.cuda.empty_cache()
gc.collect()
hidden = get_mlp([X.shape[1],  64], nn.ReLU)
normal = NormalParameter(64, numb_embeddings)
h_to_z = nn.Softmax()
embedding = nn.Embedding(X.shape[1], 50)
# p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
# out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

# embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
# embedding.weight.requires_grad=False
embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
embedding.weight.requires_grad=False
topics = EmbTopic(embedding = embedding,
                  k = numb_embeddings, normalize = False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = GSM(hidden = hidden,
            normal = normal,
            h_to_z = h_to_z,
            topics = topics,
            penalty = 0
        
            ).to(device).float()
# larger hidden size make topics more diverse
#num_docs_train = 996318
batch_size = 256
optimizer = optim.Adam(model.parameters(), 
                       lr=0.002, 
                       weight_decay=1.2e-6)



epochs = 20
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

for epoch in range(epochs):
    train(model, X,  batch_size, epoch, optimizer, scheduler)




/home/ec2-user/SageMaker/github/aspect_topic_modeling/models/NVDM.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = self.h_to_z(z)


217.4660633632115 0.3990607889635222
217.10845238821847 0.3413937655942781
216.30051204136439 0.4339985890047891
215.06954792567663 0.8085194570677621
213.8268165588379 1.0948167613574438
212.7809317452567 1.3196682802268438
211.70422199794226 1.4821047612598963
210.70782198224748 1.6934643260070257
209.74313735961914 1.9047679560525077
208.99157987322127 1.973192376749856
208.32663508823939 2.115339517593384
207.69187600272042 2.235893888132913
207.15857751028878 2.372569982494627
206.86663436889648 2.4393067998545512
206.5529294695173 2.4425294484410967
206.3272329057966 2.4712788505213603
206.23358481270927 2.480587669781276
206.1489655630929 2.534112742968968
206.18519701276506 2.5088083148002625
206.08026776994978 2.518525685582842


In [884]:
#prior(softmax_top, indexes)
emb = model.topics.get_topics().cpu().detach().numpy()
topics =  [[index_track[ind] for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(20)]
data_batch = torch.from_numpy(X.toarray()).float()
model.cpu()
h = model.hidden(data_batch)
h = model.drop(h)
mu, log_sigma = model.normal(h)
z = model.h_to_z(mu)
labels_pred = torch.argmax(z, 1).numpy()
labels_true = df['class'].values
diversity_score = np.mean(diversity(topics))
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(z.detach().numpy())
top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)  


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[94, 5, 390, 51, 3477, 135, 171, 12, 251, 23, 82, 19, 7, 2, 1, 1]
[27, 1088, 114, 466, 531, 78, 747, 405, 204, 337, 267, 66, 269, 19, 27, 9, 29, 21, 11, 6]


(0.6625035082795397,
 0.34369873489030883,
 0.6625035082795397,
 0.21958569048131982)

In [887]:
np.mean(diversity(topics))

0.26200000000000007

In [491]:
sys.path.insert(1, '/home/ec2-user/SageMaker/github/aspect_topic_modeling/pt-avitm')
from ptavitm.sklearn_api import ProdLDATransformer

In [545]:
model = ProdLDATransformer(
        cuda=device,
        batch_size=256,
        epochs=80,
        hidden1_dimension=64,
        hidden2_dimension=64,
        topics=20,
        lr=0.001,
        samples=50000)

In [546]:
model.fit(X)

In [547]:
result = model.transform(X)

In [548]:
        decoder_weight = model.autoencoder.decoder.linear.weight.detach().cpu()
        id2word = {index: str(index) for index in range(X.shape[1])}
        topics = [
            [item.item() for item in topic]
            for topic in decoder_weight.topk(min(model.score_num, X.shape[1]), dim=0)[
                1
            ].t()
        ]
 

In [549]:
diversity(topics)

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8571428571428571,
 1.0,
 1.0,
 0.8571428571428571,
 1.0,
 0.7142857142857143,
 1.0,
 1.0,
 1.0,
 0.8571428571428571,
 1.0,
 0.5714285714285714]

In [556]:
labels_pred = result.argmax(1)
labels_true = df['class'].values
diversity_score = np.mean(diversity(topics))
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
kmeans = KMeans(n_clusters=20, random_state=0).fit(normalize(result, axis=1))
top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)  


[103, 398, 243, 70, 225, 448, 265, 437, 53, 331, 336, 245, 384, 105, 182, 264, 100, 240, 344, 151]
[98, 510, 181, 339, 225, 577, 357, 313, 315, 596, 137, 452, 479, 266, 161, 241, 251, 191, 104, 173]


(0.2612756022498143,
 0.24761351519096145,
 0.3165658495171389,
 0.3046968220454173)

In [558]:
from gensim.matutils import Sparse2Corpus
corpus = Sparse2Corpus(X, documents_columns=False)


In [560]:
topics

[[1877, 1617, 2048, 1968, 1881, 1878, 1754],
 [835, 2095, 697, 1628, 2010, 711, 1833],
 [1370, 405, 1948, 1464, 2225, 1413, 2162],
 [2040, 2031, 1740, 215, 2033, 2032, 254],
 [1630, 910, 1076, 1935, 1938, 2168, 245],
 [208, 144, 1598, 1593, 1597, 1243, 1127],
 [2323, 1158, 2246, 830, 2002, 2014, 1210],
 [763, 764, 1336, 1567, 758, 462, 2159],
 [1112, 2272, 1346, 1100, 1239, 567, 1099],
 [2386, 2392, 2388, 2391, 2382, 2381, 2389],
 [1638, 1670, 1634, 1269, 2286, 166, 1671],
 [1461, 1460, 1427, 667, 1462, 1439, 1099],
 [557, 1614, 2157, 2377, 2345, 334, 1826],
 [436, 435, 438, 2175, 1034, 2258, 1986],
 [681, 1193, 2034, 599, 2239, 807, 1859],
 [1042, 1215, 1146, 1086, 1894, 1043, 1275],
 [772, 778, 1315, 774, 187, 797, 2223],
 [1742, 2088, 1744, 2039, 1562, 1745, 446],
 [1544, 1352, 417, 111, 1405, 335, 1068],
 [1562, 1542, 435, 91, 425, 438, 1541]]

In [561]:

corpus = Sparse2Corpus(X, documents_columns=False)
#decoder_weight = self.autoencoder.decoder.linear.weight.detach().cpu()
id2word = {index: str(index) for index in range(X.shape[1])}
topics = [
            [item.item() for item in topic]
            for topic in decoder_weight.topk(min(model.score_num, X.shape[1]), dim=0)[
                1
            ].t()
        ]
 
cm = CoherenceModel(
    topics=topics,
    corpus=corpus,
    dictionary=Dictionary.from_corpus(corpus, id2word),
    coherence="u_mass",
)



In [585]:
cm = CoherenceModel(
    topics=topics,
    texts = df.apply(lambda x:[str(i) for i in x['index_num']], axis = 1).values,
    corpus=corpus,
    dictionary=Dictionary.from_corpus(corpus, id2word),
    coherence='c_npmi',
)

cm.get_coherence()

-0.04799074373478669

In [586]:
cm.get_coherence_per_topic()

[0.010575433373172134,
 -0.08864524531280475,
 -0.29738757225699636,
 -0.12675700414382224,
 -0.00046802004933244707,
 -0.2907925052459905,
 -0.24912661770649866,
 0.09824602830557984,
 -0.07246353833620021,
 0.5782802533523147,
 -0.08915202093578214,
 0.19697654636731932,
 -0.209023759286811,
 -0.09121954098799176,
 -0.02777727205228651,
 0.039474022414436885,
 -0.0730726645682248,
 -0.036998336957972625,
 -0.09971504504233546,
 -0.13076801562550727]

TypeError: 'numpy.ndarray' object is not callable

In [485]:
class HiddenToLogNormal(nn.Module):
    def __init__(self, hidden_size, num_topics):
        super().__init__()
        self.fcmu = nn.Linear(hidden_size, num_topics)
        self.fclv = nn.Linear(hidden_size, num_topics)
        self.bnmu = nn.BatchNorm1d(num_topics)
        self.bnlv = nn.BatchNorm1d(num_topics)

    def forward(self, hidden):
        mu = self.bnmu(self.fcmu(hidden))
        lv = self.bnlv(self.fclv(hidden))
        dist = LogNormal(mu, (0.5 * lv).exp())
        return dist

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [488]:
from torch.distributions import kl_divergence
class NTM(nn.Module):
    """NTM that keeps track of output
    """
    def __init__(self, hidden, normal, h_to_z, topics):
        super(NTM, self).__init__()
        self.hidden = hidden
        self.normal = normal
        self.h_to_z = h_to_z
        self.topics = topics
        self.output = None
        self.drop = nn.Dropout(p=0.5)
    def forward(self, x, n_sample=1):
        h = self.hidden(x)
        h = self.drop(h)
        posterior = self.normal(h)
        prior = standard_prior_like(posterior)
        #identify how far it is away from normal distribution
        kld = kl_divergence(posterior, prior).mean()
        #print(kld.shape)
        rec_loss = 0
        for i in range(n_sample):
            #reparametrician trick
            z = posterior.rsample().to(device)
            #decode
            
            z = z / z.sum(1, keepdim=True)
            self.output = z
            #print(z)
            #z = self.drop(z)
            #get log probability for reconstruction loss
            log_prob = self.topics(z)
            rec_loss = rec_loss - (log_prob * x).sum(dim=-1)
        #average reconstruction loss
        rec_loss = rec_loss / n_sample
        #print(rec_loss.shape)
        minus_elbo = rec_loss + kld

        return {
            'loss': minus_elbo,
            'minus_elbo': minus_elbo,
            'rec_loss': rec_loss,
            'kld': kld
        }

    def get_topics(self):
        return self.topics.get_topics()



In [489]:
import gc
from torch.distributions import LogNormal, Dirichlet
#labels = [[120], [1527], [1646], [2047], [727], [1624], [36], [32], [26], [92], [907], [652]]
numb_embeddings = 20
torch.cuda.empty_cache()
gc.collect()
hidden = get_mlp([X.shape[1],  64], nn.ReLU)
normal = HiddenToLogNormal(64, numb_embeddings)
h_to_z = nn.Softmax()
embedding = nn.Embedding(X.shape[1], 50)
# p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
# out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

# embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
# embedding.weight.requires_grad=False
embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
embedding.weight.requires_grad=False
topics = EmbTopic(embedding = embedding,
                  k = numb_embeddings, normalize = False)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = NTM(hidden = hidden,
            normal = normal,
            h_to_z = h_to_z,
            topics = topics
            
        
            ).to(device).float()
# larger hidden size make topics more diverse
#num_docs_train = 996318
batch_size = 256
optimizer = optim.Adam(model.parameters(), 
                       lr=0.002, 
                       weight_decay=1.2e-6)



epochs = 20
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.002, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

for epoch in range(epochs):
    train(model, X,  batch_size, epoch, optimizer, scheduler)




nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan


KeyboardInterrupt: 

In [471]:
#prior(softmax_top, indexes)
emb = model.topics.get_topics().cpu().detach().numpy()
topics =  [[index_track[ind] for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(20)]
data_batch = torch.from_numpy(X.toarray()).float()
model.cpu()
h = model.hidden(data_batch)
h = model.drop(h)
mu = model.normal(h)
z = mu.mean
labels_pred = torch.argmax(z, 1).numpy()
labels_true = df['class'].values
diversity_score = np.mean(diversity(topics))
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(z.detach().numpy())
top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)  


[102, 102, 111, 617, 85, 52, 81, 56, 26, 27, 79, 28, 65, 76, 41, 84, 41, 53, 16, 24]
[176, 68, 63, 524, 352, 180, 109, 16, 28, 12, 3, 6, 4, 3, 4, 2, 6, 3, 5, 6]


(0.09370688740316248,
 0.031428944734180744,
 0.08330680250451024,
 0.03507299395058063)

In [472]:
import collections
collections.Counter(labels_pred)

Counter({10: 794,
         18: 924,
         7: 1669,
         17: 5989,
         14: 1185,
         9: 725,
         15: 1019,
         11: 470,
         6: 211,
         13: 292,
         4: 831,
         5: 371,
         0: 643,
         2: 914,
         19: 441,
         8: 1096,
         16: 523,
         12: 351,
         1: 195,
         3: 203})

# AG News

In [46]:
D = pd.read_csv('https://raw.githubusercontent.com/yumeng5/WeSTClass/master/agnews/dataset.csv', names = ['class', 'documents', 'text'])

In [51]:
D['clean_text'] = D['documents'] + D['text']

In [56]:
D['clean_text'] = D.swifter.apply(lambda x: ' '.join(remove_stopWords(x['documents'] + x['text'])).split(), axis=1)


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

In [57]:
#generate word 2 vec models
w2vmodel = Word2Vec(D['clean_text'].values,vector_size=200, window=10, negative = 5, min_count= 10)

#generate input for the model
vocab = list(set([j for i in init_docs for j in i if j in w2vmodel.wv]))
vocab = [''] + vocab
word_track = {i: ind for ind, i in enumerate(vocab)}
index_track = {ind: i for ind, i in enumerate(vocab)}
#pad the input


# RCV

In [58]:
from sklearn.datasets import fetch_rcv1
rcv1 = fetch_rcv1()

In [ ]:
#cannot use

In [ ]:
#backup dataset NYT and Arxiv

#https://github.com/yumeng5/JoSH/tree/master/datasets
